In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 9, 1)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-09-01
yesterday: 2023-08-31


### Tables in the process

In [3]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}', 
    
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    }

In [4]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
sells = pd.read_sql(sql, conpf)
sells.style.format(format_dict)

,name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
0,WHART,172,2022-05-17,"10,000",10.50,1052,1177,2023-08-31,10.60,219.42,15.36,"105,765.22",471,532.66,0.51%,0.39%,-52,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [6]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = sells['name']
name = names.to_string(index=False)
name

'WHART'

In [6]:
sr_qty = sells['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = sells['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = sells['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = sells['buy_id']
buy_id = buy_ids.to_string(index=False)

sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = %s
"""
sql = sql % buy_id
print(sql)

buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
WHERE B.id = 1177



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,WHART,1177,172,2022-05-17,"10,000",10.50,217.35,15.21,Sold,"105,232.56",DOS,None


In [7]:
transaction, name, qty, sell_price, buy_price

('S', 'WHART', -10000, 10.6, 10.5)

### In case of buy, retreive buy id from latest buy record (id)

In [7]:
sql = """
SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1
"""
print(sql)
buys = pd.read_sql(sql, conpf)
buys.style.format(format_dict)


SELECT name, B.* 
FROM buys B
JOIN stocks T 
ON B.stock_id = T.id
ORDER BY id DESC
LIMIT 1



,name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
0,JMART,1281,158,2023-09-01,"1,200",23.50,58.37,4.09,Active,"28,262.46",DTD,None


In [8]:
transaction = 'B'

names = buys['name']
name = names.to_string(index=False)

sr_qty = buys['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty)

sr_price = buys['price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

transaction, name, qty, buy_price

('B', 'JMART', 1200, 23.5)

## Begin of Sale transaction

### Begin of Update buy table in MySQL stock database from sale transaction

In [9]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'JMART'



,name,date,shares,price,active,period,grade
0,JMART,2023-01-20,"10,800",8.50,1,4,C3


In [10]:
qty = -4800
new_cost = 34.5
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + -4800, price = 34.5
WHERE name = 'JMART'



In [11]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [12]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'JMART'



,name,date,shares,price,active,period,grade
0,JMART,2023-01-20,"6,000",34.50,1,4,C3


### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [14]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'JMART'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,JMART,0.6600,0.0000,0.0000,0.0000,0.6600,"8,400",2023-08-24,2023-09-08,,0


In [15]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + 1200
WHERE name = 'JMART'



In [16]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [17]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'JMART'



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,JMART,0.6600,0.0000,0.0000,0.0000,0.6600,"9,600",2023-08-24,2023-09-08,,0


### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [18]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'JMART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,111,JMART,0.00,0.00,B,22.50,24.70,0.00,0.00,34.50,"1,200",-4,4,"7,200",0.000000,0.000000,RD35pct,SET50


In [19]:
buy_target  = 22.5 # must calculate manually beforehand
sell_target = 24.7
cost = 34.5
status = 'B'
reason = "RD35pct"
sell_qty = 1200

sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s,
qty = %s, buy_target = %s, sell_target = %s, cost = %s, reason = '%s', status = '%s'
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, sell_qty, buy_target, sell_target, cost, reason, status, name)
print(sqlUpd)


UPDATE stocks
SET available_qty = available_qty + 1200,
qty = 1200, buy_target = 22.5, sell_target = 24.7, cost = 34.5, reason = 'RD35pct', status = 'B'
WHERE name = 'JMART'



In [20]:
rp = conlite.execute(sqlUpd)
rp.rowcount

1

In [21]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * 
FROM stocks 
WHERE name = 'JMART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,111,JMART,0.00,0.00,B,22.50,24.70,0.00,0.00,34.50,"1,200",-4,4,"8,400",0.000000,0.000000,RD35pct,SET50


### End of Update stocks table in SQLite port_lite database from sale transaction

## End of Sale process

## Begin of  Buy process

### Begin of Update buy table in MySQL database from buy transaction

In [22]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy['date'] = pd.to_datetime(buy['date'])
buy.drop(['volsell', 'volbal'], axis=1, inplace = True)
buy.rename(columns={'volbuy':'shares'}, inplace = True)
buy['shares'] = buy['shares'].astype('int64')
buy


SELECT * 
FROM buy 
WHERE name = 'JMART'



,name,date,shares,price,active,dividend,period,grade
0,JMART,2023-01-20,9600,34.5,1,0.66,4,C3


### In case there is the same stock name in port

In [23]:
#qty = 1200
new_cost = 8.50 # new average cost must be calculated manually beforehand
sqlUpd = """
UPDATE buy
SET volbuy = volbuy + %s, price = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, name)
print(sqlUpd)


UPDATE buy
SET volbuy = volbuy + 1200, price = 8.5
WHERE name = 'JMART'



In [24]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [25]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'JMART'



,name,date,shares,price,active,dividend,period,grade
0,JMART,2023-01-20,"10,800",8.50,1,0.6600,4,C3


### In case of totally new stock

In [26]:
rcd = ['AIMIRT', '2023-08-17', 10000, 11, 0, 0, 1, 0, 2,'A3']
rcd

['AIMIRT', '2023-08-17', 10000, 11, 0, 0, 1, 0, 2, 'A3']

In [27]:
sqlIns = """
INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)


INSERT INTO buy (name, date, volbuy, price, volsell, volbal, active, dividend, period, grade)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)



In [28]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

1

In [29]:
sql = """
SELECT * 
FROM buy 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

buy = pd.read_sql(sql, const)
buy.drop(['volsell', 'volbal'], axis=1, inplace=True)
buy.rename(columns={'volbuy':'shares'},inplace=True)
buy['shares'] = buy['shares'].astype('int64')
buy.style.format(format_dict)


SELECT * 
FROM buy 
WHERE name = 'JMART'



,name,date,shares,price,active,dividend,period,grade
0,JMART,2023-01-20,"10,800",8.50,1,0.6600,4,C3


### End of Update buy table in MySQL database from buy transaction

### Begin of Update dividend table in MySQL stock database from buy transaction

In [30]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'JMART' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,JMART,0.6600,0.0000,0.0000,0.0000,0.6600,"9,600",2023-08-24,2023-09-08,,0


In [31]:
sqlUpd = """
UPDATE dividend
SET shares = shares + %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, name)
print(sqlUpd)


UPDATE dividend
SET shares = shares + 1200
WHERE name = 'JMART'



In [32]:
rp = const.execute(sqlUpd)
rp.rowcount

1

In [33]:
sql = """
SELECT * 
FROM dividend 
WHERE name = '%s' 
"""
sql = sql % name
print(sql)

dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)


SELECT * 
FROM dividend 
WHERE name = 'JMART' 



,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,JMART,0.6600,0.0000,0.0000,0.0000,0.6600,"10,800",2023-08-24,2023-09-08,,0


### In case of totally new stock

In [34]:
rcd = ['ASK', 1.44, 0, 0, 0, 1.44, 4500, '2023-02-28', '2023-04-27', '', 1]
rcd

['ASK', 1.44, 0, 0, 0, 1.44, 4500, '2023-02-28', '2023-04-27', '', 1]

In [35]:
sqlIns = """
INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
print(sqlIns)


INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)



In [36]:
rp = const.execute(sqlIns, rcd)
rp.rowcount

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry 'ASK' for key 'PRIMARY'")
[SQL: 
INSERT INTO dividend (name, q4, q3, q2, q1, dividend, shares, xdate, paiddate, kind, actual)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
]
[parameters: ('ASK', 1.44, 0, 0, 0, 1.44, 4500, '2023-02-28', '2023-04-27', '', 1)]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

### End of Create dividend table in MySQL stock database from buy transaction

### Begin of Update stocks table in SQLite port_lite database from buy transaction

In [ ]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks

In [ ]:
new_cost = 8.5
buy_target = 6.9
sell_target = 7.4
status = 'I'
reason = 'DOS'
new_buy_qty = 6000
new_sell_qty = 6000
new_cost, buy_target, sell_target, status, reason, new_buy_qty, new_sell_qty 

In [ ]:
#qty = 12000
sqlUpd = """
UPDATE stocks
SET available_qty = available_qty + %s, 
cost = %s, buy_target = %s, sell_target = %s, status = '%s', 
reason = '%s', qty = %s
WHERE name = '%s'
"""
sqlUpd = sqlUpd % (qty, new_cost, buy_target, sell_target, status, reason, new_sell_qty, name)
print(sqlUpd)

In [ ]:
rp = conlite.execute(sqlUpd)
rp.rowcount

In [ ]:
sql = """
SELECT * 
FROM stocks 
WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)

### End of Update stocks table in SQLite port_lite database from buy transaction

## End of  Buy process

In [ ]:
sql = "SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, \
FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,\
FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, \
FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, \
FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol \
FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '%s' \
ORDER BY period, buy.name"
sql = sql % (yesterday)
print(sql)

In [ ]:
output = pd.read_sql(sql, const)
output

In [ ]:
file_name = 'portfolio.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

output.to_csv(output_file, header=True, index=False)
output.to_csv(data_file, header=True, index=False)
output.to_csv(one_file, header=True, index=False)
output.to_csv(osd_file, header=True, index=False)